In [1]:
import numpy as np
from scipy.linalg import block_diag
import matplotlib.pyplot as plt
import pickle
from stochastic_cramer_rao import cramer_rao

# Number of sensors
M = 12

# Number of nodes
K = 6

# Locations of the nodes with respect to initial node
locations = (np.array([0,0]),np.array([0.45,0.99]),np.array([3.02,0.45]),np.array([5.61,0.90])
             ,np.array([8.03,1.46]),np.array([8.70,0.50]))

# Direction of arrivals
angles = np.array([5,-14,-10])*2*(np.pi)/360
K_l = np.array([np.sin(angles), np.cos(angles)])

# Steering vector is implemented. It should have the shape: (12,3)
for j in range(0,len(locations)):
    for i in range(0,len(angles)):
        first_part = np.exp(1j*np.pi*locations[j].dot(K_l[:,i]))
        second_part = np.array([1, np.exp(1j*np.pi*1*np.sin(angles[i]))])
        A_k = np.transpose(np.array(first_part*second_part, ndmin=2))
        if i == 0:
            B_k = A_k
        else:
            B_k = np.concatenate((B_k, A_k), axis=1)
        
        if i == len(angles)-1:
            if j == 0:
                A = np.transpose(B_k)
            else:
                A = np.concatenate((A, np.transpose(B_k)), axis=1)

A = np.transpose(A)

MSE = np.zeros(80)
cramer = np.zeros(80)

# uncomment here and comment for N_samples if you want to iterate over SNR
# MSE = np.zeros(80)
for x in range(0,7):
    MSE = np.zeros(80)
    for snr_dB in range(-20,60):
    #for N_samples in range(2,1002):
        for i in range(500):

            #snr_dB = 10
            # Signal(A*s) to noise(n) ratio
            received_snr = 10**(snr_dB/20)
            ratio_As_to_s = 1/3
            snr = received_snr*ratio_As_to_s
            N_samples = 100
            #snr = 10**(snr_dB/20)

            # Source signal implementation (shape: (3,500))
            signal = np.random.normal(0,np.sqrt(snr),(3,N_samples))
            #w = np.atleast_2d([np.pi/3, np.pi/4, np.pi/5]).T
            #signal = (np.sqrt(snr/3))*np.exp(1j*w*(np.atleast_2d(np.arange(1,N_samples+1))))

            # Received signal power on sensors
            signal_power = sum(sum(np.abs(A.dot(signal))**2))

            # Noise signal implementation (shape: (12,500))
            noise = np.random.normal(0,np.sqrt(0.5),(12,N_samples)) + 1j*np.random.normal(0,np.sqrt(0.5),(12,N_samples))
            noise_power  = sum(sum(np.abs(noise)**2))
            if i == 0:
                print("SIGNAL TO NOISE RATIO")
                print(signal_power/noise_power)

            # Received signal (shape: (12,500))
            z = A.dot(signal) + noise

            # Sample covariance matrix
            R_sample = z.dot(z.conj().T)

            # Eigenvalue and eigenvectors
            w_sample, v_sample = np.linalg.eig(R_sample)
            if i == 0 and snr_dB == -20:
                print()
                print("EIGENVALUES OF SAMPLE COVARIANCE MATRIX")
                print(w_sample[0])
                print(w_sample[1])
                print(w_sample[2])
                print(w_sample[3])

            # Sensor Selection Matrix (shape: (12,6))
            T = np.array([[1,0,0,0,0,0],
                          [1,0,0,0,0,0],
                          [0,1,0,0,0,0],
                          [0,1,0,0,0,0],
                          [0,0,1,0,0,0],
                          [0,0,1,0,0,0],
                          [0,0,0,1,0,0],
                          [0,0,0,1,0,0],
                          [0,0,0,0,1,0],
                          [0,0,0,0,1,0],
                          [0,0,0,0,0,1],
                          [0,0,0,0,0,1]])

            # Push-Sum Matrix (shape: (6,6))
            P_push = np.array([[0.2,0.2,0.2,0  ,0  ,0],
                          [0.2,0.2,0.2,0  ,0  ,0],
                          [0.6,0.6,0.2,0.2,0  ,0],
                          [0  ,0  ,0.4,0.2,0.2,0.2],
                          [0  ,0  ,0  ,0.2,0.2,0.2],
                          [0  ,0  ,0  ,0.4,0.6,0.6]])

            # Average-Consensus Matrix (shape: (6,6))
            P_ave = np.array([[0.5,0.25,0.25,0  ,0  ,0],
                          [0.25,0.5,0.25,0  ,0  ,0],
                          [0.25,0.25,0.25,0.25,0  ,0],
                          [0  ,0  ,0.25,0.25,0.25,0.25],
                          [0  ,0  ,0  ,0.25,0.5,0.25],
                          [0  ,0  ,0  ,0.25,0.25,0.5]])

            # Weight Vector  (shape: (6,1))
            w = np.atleast_2d([1,1,1,1,1,1]).T

            if x == 0:
                # Push-Sum Covariance Matrix Estimation
                R_push_numerator = np.multiply(T.dot(np.linalg.matrix_power(P_push,10)).dot(T.T), R_sample)
                R_push_denominator = T.dot(np.linalg.matrix_power(P_push,10)).dot(w).dot(np.ones((1,6))).dot(T.T)

                # Push Sum Covariance Matrix (shape: (12,12))
                R_push = K*np.multiply(R_push_numerator, (1/(R_push_denominator)))
                R = R_push
            if x == 1:
                # Push-Sum Covariance Matrix Estimation
                R_push_numerator = np.multiply(T.dot(np.linalg.matrix_power(P_push,20)).dot(T.T), R_sample)
                R_push_denominator = T.dot(np.linalg.matrix_power(P_push,20)).dot(w).dot(np.ones((1,6))).dot(T.T)

                # Push Sum Covariance Matrix (shape: (12,12))
                R_push = K*np.multiply(R_push_numerator, (1/(R_push_denominator)))
                R = R_push
            if x == 2:
                # Push-Sum Covariance Matrix Estimation
                R_push_numerator = np.multiply(T.dot(np.linalg.matrix_power(P_push,30)).dot(T.T), R_sample)
                R_push_denominator = T.dot(np.linalg.matrix_power(P_push,30)).dot(w).dot(np.ones((1,6))).dot(T.T)

                # Push Sum Covariance Matrix (shape: (12,12))
                R_push = K*np.multiply(R_push_numerator, (1/(R_push_denominator)))  
                R = R_push

            if x == 3:
                # Average Consensus Covariance Matrix Estimation      
                R_ave_con = K * np.multiply(T.dot(np.linalg.matrix_power(P_ave,10)).dot(T.T), R_sample)
                R = R_ave_con

            if x == 4:
                # Average Consensus Covariance Matrix Estimation      
                R_ave_con = K * np.multiply(T.dot(np.linalg.matrix_power(P_ave,20)).dot(T.T), R_sample)
                R = R_ave_con         

            if x == 5:
                # Average Consensus Covariance Matrix Estimation      
                R_ave_con = K * np.multiply(T.dot(np.linalg.matrix_power(P_ave,30)).dot(T.T), R_sample)
                R = R_ave_con

            if x == 6:
                # Conventional ESPRIT Algorithm      
                R = R_sample               

    #         if i == 0 and snr_dB == -20:
    #             print()
    #             print("                            R_push")
    #             print("****************************************************************")
    #             print("****************************************************************")
    #             print(R_push)
    #             print()
    #             print("                           R_sample")
    #             print("****************************************************************")
    #             print("****************************************************************")
    #             print(R_sample)
    #             print()
    #             # Check Convergence Status
    #             print(R_push == R_sample)

            w_push, v_push = np.linalg.eig(R)

            # Upper group selection matrix J_up
            J_up = np.kron(np.eye(6),np.array([1,0]))

            # Lower group selection matrix J_down
            J_down = np.kron(np.eye(6),np.array([0,1]))

            # Push-Sum estimated signal eigenvector matrices
            U_s_push = v_push[:,:3]

            # Upper signal eigenvectors
            U_s_up = J_up.dot(U_s_push)

            # Lower signal eigenvectors
            U_s_down = J_down.dot(U_s_push)

            # Matrix including knowledge about DOAs of the source signals
            psi = np.linalg.inv((U_s_up.conj().T).dot(U_s_up)).dot((U_s_up.conj().T)).dot(U_s_down)

            w2, v2 = np.linalg.eig(psi)
            doa = []
            doa.append(np.arcsin(np.angle(w2[0])/np.pi)/(2*np.pi)*360)
            doa.append(np.arcsin(np.angle(w2[1])/np.pi)/(2*np.pi)*360)
            doa.append(np.arcsin(np.angle(w2[2])/np.pi)/(2*np.pi)*360)

            if i == 0:
                print()
                print("  DOAs of the source signals in degrees with SNR: " + str(snr_dB) )
                print("  DOAs of the source signals in degrees with N_samples: " + str(N_samples) )
                print("****************************************************************")
                print("****************************************************************")
                print("DOA of the first source signal:   " + str(doa[0]))
                print("DOA of the second source signal:   " + str(doa[1]))
                print("DOA of the third source signal:   " + str(doa[2]))

            # Derivation matrix
            D = np.vstack((np.gradient(A[:,0]), np.gradient(A[:,1]), np.gradient(A[:,2]))).T
            # Orthogonal complement of the projection matrix
            P_A = np.identity(12) - A.dot(np.linalg.inv(A.conj().T.dot(A))).dot(A.conj().T)
            # Emitter covariance matrix
            S = signal.dot(signal.conj().T)

            #print(np.real(D.conj().T.dot(P_A).dot(D)).shape)
            #print((S.dot(A.conj().T)))
            #print((S.dot(A.conj().T).dot(np.linalg.inv(R_sample))))
            #print((S.dot(A.conj().T).dot(np.linalg.inv(R_sample)).dot(A)))
            #print(((S.dot(A.conj().T).dot(np.linalg.inv(R_sample)).dot(A).dot(S)).T).shape)
            diff_1 = min(abs(doa[0]-angles*360/(2*np.pi)))
            diff_2 = min(abs(doa[1]-angles*360/(2*np.pi)))
            diff_3 = min(abs(doa[2]-angles*360/(2*np.pi)))

            # Uncomment here if you want to iterate over SNR
            # Here in every iteration doa angles change and we take 1/500 times of it. This corresponds to Monte-Carlo simulation.
            
            MSE[snr_dB+20] = MSE[snr_dB+20] + 1/3*1/500*((diff_1)**2+(diff_2)**2+(diff_3)**2)
            # MSE[N_samples-2] = MSE[N_samples-2] + 1/3*1/500*((diff_1)**2+(diff_2)**2+(diff_3)**2)
            # cramer[N_samples-2] = 1/(2*N_samples)*np.abs(np.trace(1/(np.multiply((np.real(D.conj().T.dot(P_A).dot(D))),((S.dot(A.conj().T).dot(np.linalg.inv(R_sample)).dot(A).dot(S)).T)))))*360/(2*np.pi)
            # cramer_rao[N_samples-2] = 1/(2*N_samples)*np.abs(np.trace(np.linalg.pinv(np.real(np.multiply(((D.conj().T.dot(P_A).dot(D))),((S.dot(A.conj().T).dot(np.linalg.inv(R_sample)).dot(A).dot(S)).T))))))*360/(2*np.pi)
            # cramer_rao[N_samples-2] = 1/(2*N_samples)*np.abs(np.trace(np.linalg.pinv(np.multiply((np.real(D.conj().T.dot(P_A).dot(D))),(S.T)))))*360/(2*np.pi)
            # cramer[snr_dB+20] = cramer[snr_dB+20] + 1/500*cramer_rao(A, S, N_samples, angles, locations)
            # cramer[N_samples-2] = cramer[N_samples-2] + 1/500*cramer_rao(A, signal, angles, locations)
            # cramer[snr_dB+20] = cramer[snr_dB+20] + 1/500*cramer_rao(A, signal, angles, locations)
            if i == 499: 
                print("MSE")
                print(MSE[snr_dB+20])
                #print(MSE[N_samples-2])
    
    RMSE = np.sqrt(MSE)
    # load the results in memory since it takes too much time to obtain it again.(1000 iteration for N=1000)
    # (Don't forget to change the name when you change some parameters above)
    if x == 0:
        with open("Push_Sum_ESPRIT_SNR_10dB_P_10.txt", "wb") as fp:   #Pickling
            pickle.dump(RMSE, fp)
    if x == 1:
        with open("Push_Sum_ESPRIT_SNR_10dB_P_20.txt", "wb") as fp:   #Pickling
            pickle.dump(RMSE, fp)    
    if x == 2:
        with open("Push_Sum_ESPRIT_SNR_10dB_P_30.txt", "wb") as fp:   #Pickling
            pickle.dump(RMSE, fp)
    if x == 3:
        with open("Average_Consensus_ESPRIT_SNR_10dB_P_10.txt", "wb") as fp:   #Pickling
            pickle.dump(RMSE, fp)
    if x == 4:
        with open("Average_Consensus_ESPRIT_SNR_10dB_P_20.txt", "wb") as fp:   #Pickling
            pickle.dump(RMSE, fp)
    if x == 5:
        with open("Average_Consensus_ESPRIT_SNR_10dB_P_30.txt", "wb") as fp:   #Pickling
            pickle.dump(RMSE, fp)
    if x == 6:
        with open("Conventional_ESPRIT_SNR_10dB.txt", "wb") as fp:   #Pickling
            pickle.dump(RMSE, fp)


SIGNAL TO NOISE RATIO
0.10186901007553943

EIGENVALUES OF SAMPLE COVARIANCE MATRIX
(191.43553358866814-2.2128892538438538e-16j)
(172.10831462777293+3.644493163557078e-15j)
(44.96501898952237+3.599146958979993e-15j)
(154.16818153873692+4.458355197353287e-15j)

  DOAs of the source signals in degrees with SNR: -20
  DOAs of the source signals in degrees with N_samples: 100
****************************************************************
****************************************************************
DOA of the first source signal:   9.779292556929219
DOA of the second source signal:   -3.6234921082211127
DOA of the third source signal:   -37.78494283144128
MSE
455.7326832701667
SIGNAL TO NOISE RATIO
0.11590025874066404

  DOAs of the source signals in degrees with SNR: -19
  DOAs of the source signals in degrees with N_samples: 100
****************************************************************
****************************************************************
DOA of the first source sig

MSE
4.541195018369418
SIGNAL TO NOISE RATIO
0.7881532287325596

  DOAs of the source signals in degrees with SNR: -2
  DOAs of the source signals in degrees with N_samples: 100
****************************************************************
****************************************************************
DOA of the first source signal:   4.013184037049173
DOA of the second source signal:   -10.660295690117792
DOA of the third source signal:   -8.58428539206293
MSE
4.039900950471636
SIGNAL TO NOISE RATIO
0.9973142145612772

  DOAs of the source signals in degrees with SNR: -1
  DOAs of the source signals in degrees with N_samples: 100
****************************************************************
****************************************************************
DOA of the first source signal:   3.0112308598957167
DOA of the second source signal:   -13.727517285719246
DOA of the third source signal:   -10.26176076387788
MSE
3.5914699584134913
SIGNAL TO NOISE RATIO
1.0086890740101682

 

MSE
1.5666051956838405
SIGNAL TO NOISE RATIO
5.65770731122505

  DOAs of the source signals in degrees with SNR: 16
  DOAs of the source signals in degrees with N_samples: 100
****************************************************************
****************************************************************
DOA of the first source signal:   4.709848704688006
DOA of the second source signal:   -12.431599123611107
DOA of the third source signal:   -7.521183845632486
MSE
1.5384979992778047
SIGNAL TO NOISE RATIO
6.735977564543695

  DOAs of the source signals in degrees with SNR: 17
  DOAs of the source signals in degrees with N_samples: 100
****************************************************************
****************************************************************
DOA of the first source signal:   5.354555015733723
DOA of the second source signal:   -11.846247659045366
DOA of the third source signal:   -8.198183851113484
MSE
1.5591453964549165
SIGNAL TO NOISE RATIO
9.040295510579158

  D

MSE
1.4991832051545102
SIGNAL TO NOISE RATIO
41.178027029388545

  DOAs of the source signals in degrees with SNR: 34
  DOAs of the source signals in degrees with N_samples: 100
****************************************************************
****************************************************************
DOA of the first source signal:   4.77596977373755
DOA of the second source signal:   -12.343622504116091
DOA of the third source signal:   -9.846563234463584
MSE
1.5155013828666692
SIGNAL TO NOISE RATIO
52.28217236894303

  DOAs of the source signals in degrees with SNR: 35
  DOAs of the source signals in degrees with N_samples: 100
****************************************************************
****************************************************************
DOA of the first source signal:   4.824773827880661
DOA of the second source signal:   -12.27002501721447
DOA of the third source signal:   -8.822303222629206
MSE
1.536157138789094
SIGNAL TO NOISE RATIO
58.869390740134754

  D

MSE
1.5605500106050092
SIGNAL TO NOISE RATIO
376.9753835606531

  DOAs of the source signals in degrees with SNR: 52
  DOAs of the source signals in degrees with N_samples: 100
****************************************************************
****************************************************************
DOA of the first source signal:   4.949493265759949
DOA of the second source signal:   -12.20629215702537
DOA of the third source signal:   -8.926661584076406
MSE
1.565195038857647
SIGNAL TO NOISE RATIO
471.9504696681038

  DOAs of the source signals in degrees with SNR: 53
  DOAs of the source signals in degrees with N_samples: 100
****************************************************************
****************************************************************
DOA of the first source signal:   4.7509196960660995
DOA of the second source signal:   -12.241243749876718
DOA of the third source signal:   -8.905620611066471
MSE
1.550915941268927
SIGNAL TO NOISE RATIO
479.22485059091053

  D

...

MSE
345.0105606468809
SIGNAL TO NOISE RATIO
0.16308112689059595

  DOAs of the source signals in degrees with SNR: -15
  DOAs of the source signals in degrees with N_samples: 100
****************************************************************
****************************************************************
DOA of the first source signal:   -8.337511142081643
DOA of the second source signal:   -14.095899820067011
DOA of the third source signal:   18.27715245883321
MSE
309.8427819250884
SIGNAL TO NOISE RATIO
0.16541752249662514

  DOAs of the source signals in degrees with SNR: -14
  DOAs of the source signals in degrees with N_samples: 100
****************************************************************
****************************************************************
DOA of the first source signal:   -16.744357346758793
DOA of the second source signal:   13.092522420344912
DOA of the third source signal:   -0.5911045885597416
MSE
298.2844425798888
SIGNAL TO NOISE RATIO
0.27150758977849

MSE
4.342361346608296
SIGNAL TO NOISE RATIO
1.2890990406391047

  DOAs of the source signals in degrees with SNR: 3
  DOAs of the source signals in degrees with N_samples: 100
****************************************************************
****************************************************************
DOA of the first source signal:   5.073887689759777
DOA of the second source signal:   -13.368013346951887
DOA of the third source signal:   -9.479801852312022
MSE
3.3552545780970466
SIGNAL TO NOISE RATIO
1.818728878675352

  DOAs of the source signals in degrees with SNR: 4
  DOAs of the source signals in degrees with N_samples: 100
****************************************************************
****************************************************************
DOA of the first source signal:   4.239343088925383
DOA of the second source signal:   -13.539315109921796
DOA of the third source signal:   -8.886809760583645
MSE
3.507255525098121
SIGNAL TO NOISE RATIO
1.4625922410598096

  DO

MSE
0.43171759187048975
SIGNAL TO NOISE RATIO
11.142314866178028

  DOAs of the source signals in degrees with SNR: 21
  DOAs of the source signals in degrees with N_samples: 100
****************************************************************
****************************************************************
DOA of the first source signal:   4.970891335427516
DOA of the second source signal:   -10.052075139185284
DOA of the third source signal:   -14.683450362810111
MSE
0.3794403386430337
SIGNAL TO NOISE RATIO
10.866349303549983

  DOAs of the source signals in degrees with SNR: 22
  DOAs of the source signals in degrees with N_samples: 100
****************************************************************
****************************************************************
DOA of the first source signal:   4.386385721031213
DOA of the second source signal:   -13.291817301090735
DOA of the third source signal:   -12.652232810955686
MSE
0.33364970078502854
SIGNAL TO NOISE RATIO
13.333403747834

MSE
0.05423964090432563
SIGNAL TO NOISE RATIO
77.99709429250898

  DOAs of the source signals in degrees with SNR: 39
  DOAs of the source signals in degrees with N_samples: 100
****************************************************************
****************************************************************
DOA of the first source signal:   4.693929537041218
DOA of the second source signal:   -9.67497454823554
DOA of the third source signal:   -14.099109476497102
MSE
0.04514927935337762
SIGNAL TO NOISE RATIO
117.81401427734765

  DOAs of the source signals in degrees with SNR: 40
  DOAs of the source signals in degrees with N_samples: 100
****************************************************************
****************************************************************
DOA of the first source signal:   5.060485070889621
DOA of the second source signal:   -13.807834036921825
DOA of the third source signal:   -9.915598729751151
MSE
0.03949435181193625
SIGNAL TO NOISE RATIO
115.67405380624133

MSE
0.0059452019979809095
SIGNAL TO NOISE RATIO
710.6190259338518

  DOAs of the source signals in degrees with SNR: 57
  DOAs of the source signals in degrees with N_samples: 100
****************************************************************
****************************************************************
DOA of the first source signal:   5.0238066962969485
DOA of the second source signal:   -10.002583344293493
DOA of the third source signal:   -14.11891936127575
MSE
0.005788319685547485
SIGNAL TO NOISE RATIO
806.643907133255

  DOAs of the source signals in degrees with SNR: 58
  DOAs of the source signals in degrees with N_samples: 100
****************************************************************
****************************************************************
DOA of the first source signal:   5.036070823731435
DOA of the second source signal:   -10.020460169285808
DOA of the third source signal:   -14.022838131950651
MSE
0.005110419165895489
SIGNAL TO NOISE RATIO
1114.17375548